In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from typing import Literal
from pydantic import BaseModel

class Args(BaseModel):
    company: Literal["LMK", "AMK", "GL", "RT"]
    env: Literal["dev", "prod"]
    is_running_on_databricks: bool


In [19]:
args = Args(
    company="GL",
    env="dev",
    is_running_on_databricks=False
)

is_running_on_databricks = args.is_running_on_databricks

In [20]:
from databricks_env import auto_setup_env
if is_running_on_databricks:
    auto_setup_env()

In [21]:
from dishes_forecasting.train.configs.train_configs import get_training_configs
train_config = get_training_configs(company_code=args.company)

In [22]:
import logging

from constants.companies import get_company_by_code

company_code = args.company
company = get_company_by_code(company_code=company_code)
company_id = company.company_id

In [23]:
env = "dev"

In [24]:
from dishes_forecasting.train.training_set import create_training_set
from dishes_forecasting.train.configs.feature_lookup_config import feature_lookup_config_list

training_set, df_training_pk_target = create_training_set(
    company_id=company_id,
    train_config=train_config,
    feature_lookup_config_list=feature_lookup_config_list,
    is_drop_ignored_columns=True,
)

training_set

,menu_year,menu_week,variation_ratio,portion_quantity,recipe_name,cooking_time_from,cooking_time_to,recipe_difficulty_level_id,recipe_main_ingredient_id,price_category_level_id,...,has_bacon,has_wok_sauce,has_asian_sauces,has_salsa,has_flat_bread,has_pita,has_potato,has_rice,has_nuts,has_chili
0,2021,1,0.003134,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2021,1,0.004883,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021,1,0.018292,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021,1,0.029005,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021,1,0.007798,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56652,2025,32,0.000389,6.0,Konfiterte Kyllingvinger Med Frisk Eplecolesla...,20.0,30.0,2.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
56653,2025,32,0.000097,6.0,Plantebasert Hoisin Chunks Bowl Med Sesamstekt...,20.0,30.0,2.0,3.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
56654,2025,32,0.000097,5.0,Pannestekt Svinekam I Soyasjysaus Med Lun Nekt...,30.0,40.0,2.0,7.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
56655,2025,32,0.000049,5.0,"Japansk Kyllingschnitzel Med Syrlig Kålsalat, ...",30.0,40.0,1.0,5.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [25]:
import mlflow

mlflow.set_registry_uri("databricks-uc")
mlflow.set_experiment("/Shared/ml_experiments/dishes-forecasting")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/1523657300871524', creation_time=1727777979348, experiment_id='1523657300871524', last_update_time=1754573828910, lifecycle_stage='active', name='/Shared/ml_experiments/dishes-forecasting', tags={'mlflow.databricks.filesystem.experiment_permissions_check': 'test',
 'mlflow.experiment.sourceName': '/Shared/ml_experiments/dishes-forecasting',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'sylvia.liu@godtlevert.no',
 'mlflow.ownerId': '5863260687036347'}>

In [26]:
from dishes_forecasting.train.train_pipeline import train_model
from dishes_forecasting.train.configs.hyper_params import load_hyperparams
params_lgb, params_rf, params_xgb = load_hyperparams(company=args.company)

is_running_on_databricks=False

custom_pipeline, X_train, X_test, y_train, y_test, mape, mae, df_test_metrics, df_test_binned = train_model(
    training_set=training_set,
    params_lgb=params_lgb,
    params_rf=params_rf,
    params_xgb=params_xgb,
    env="dev",
    is_running_on_databricks=is_running_on_databricks,
    train_config=train_config,
    company=company,
    is_register_model=False,
    is_log_model=True
)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sylvia.liu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Forcing feature columns to be the defined data type...
Training set shape: (24211, 46)
Test set shape: (494, 46)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032415 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4799
[LightGBM] [Info] Number of data points in the train set: 24211, number of used features: 509
[LightGBM] [Info] Start training from score -6.516833
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

/Users/sylvia.liu/lmk/sous-chef2/projects/dishes-forecasting/dishes_forecasting/train/metrics.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_test.groupby("quantity_bin")
🔗 View Logged Model at: https://adb-4291784437205825.5.azuredatabricks.net/ml/experiments/1523657300871524/models/m-1dca5c680d8348229a84e3e3bfbddcde
/Users/sylvia.liu/lmk/sous-chef2/projects/dishes-forecasting/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (tra

🏃 View run GL_2025-08-07-15:48:09 at: https://adb-4291784437205825.5.azuredatabricks.net/ml/experiments/1523657300871524/runs/a2a2bf38c8da471cb0d3b6a0b0f7a79e
🧪 View experiment at: https://adb-4291784437205825.5.azuredatabricks.net/ml/experiments/1523657300871524


In [27]:
mae

17.73076923076923

In [28]:
mape

0.595645322521844

In [29]:
df_test_binned

,quantity_bin_,abs_error_mean,abs_error_median,abs_error_pct_mean,abs_error_pct_median,product_variation_quantity_count
0,0-100,10.609589,5.0,0.625780,0.400000,438
1,100-200,42.307692,36.0,0.300479,0.295652,39
2,200-300,106.583333,111.5,0.443311,0.442601,12
3,300-400,198.000000,208.5,0.569868,0.608231,4
4,400-500,391.000000,391.0,0.839056,0.839056,1
5,500-inf,NaN,NaN,NaN,NaN,0


In [16]:
X_test

,menu_year,menu_week,portion_quantity,recipe_name,cooking_time_from,cooking_time_to,recipe_difficulty_level_id,recipe_main_ingredient_id,taxonomy_list,number_of_taxonomies,...,has_bacon,has_wok_sauce,has_asian_sauces,has_salsa,has_flat_bread,has_pita,has_potato,has_rice,has_nuts,has_chili
56157,2025,31,6.0,"Vegetarpizza Med Skjørost Fra Røros, Eplesalat...",15.0,20.0,3.0,3.0,"Ekspress, Express, Pizza, Uten Svin, Chefs Cho...",8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56158,2025,31,5.0,Italiensk Linse- Og Kikertgryte Med Potetmos O...,15.0,20.0,3.0,3.0,"Ekspress, Express, Family, Fort Gjort, Glutenf...",10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56159,2025,31,5.0,"Mild Texmex-bowl Med Kjøttdeig, Ris Og Grønnsaker",15.0,20.0,3.0,8.0,"Barnevennlig, Bowl, Express, Glutenfri, Kjøtt,...",7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
56160,2025,31,3.0,Storfe- Og Linsegryte Med Grønnsaker Og Sitron...,20.0,30.0,2.0,8.0,"Laktosefri, Uten Svin, Kjøtt, Low Calorie, Stew",5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56161,2025,31,6.0,Tofu Katsu Med Ris Og Frisk Agurksalat Med Ses...,20.0,30.0,2.0,3.0,"Breaded, Laktosefri, Uten Svin, Vegetar, Veget...",6.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56652,2025,32,6.0,Konfiterte Kyllingvinger Med Frisk Eplecolesla...,20.0,30.0,2.0,5.0,"Bbq, Glutenfri, Kylling, Uten Svin",4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
56653,2025,32,6.0,Plantebasert Hoisin Chunks Bowl Med Sesamstekt...,20.0,30.0,2.0,3.0,"Asian, Bowl, Laktosefri, Vegetar, Vegetar, Veg...",9.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
56654,2025,32,5.0,Pannestekt Svinekam I Soyasjysaus Med Lun Nekt...,30.0,40.0,2.0,7.0,"Glutenfri, Kjøtt, Laktosefri, Low Calorie, Summer",5.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
56655,2025,32,5.0,"Japansk Kyllingschnitzel Med Syrlig Kålsalat, ...",30.0,40.0,1.0,5.0,"Laktosefri, Uten Svin, Asia, Chefs Choice, Kyl...",5.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [56]:
import numpy as np
y_pred_transformed = np.exp(custom_pipeline.predict(X_test))

In [ ]:
from dishes_forecasting.train.training_set import get_training_pk_target
df_training_target = get_training_pk_target(
    company_id=company_id,
    min_yyyyww=train_config["train_start_yyyyww"],
    max_yyyyww=train_config["train_end_yyyyww"],
    is_training_set=False,
)

In [9]:
from dishes_forecasting.schema import feature_schema
feature_schema.coerce=True

In [ ]:
feature_schema.validate(training_set.dropna())

In [ ]:
X_test

In [82]:
mlflow.set_tracking_uri("databricks")